In [1]:
using Pkg

Pkg.add("JuMP")
Pkg.add("GLPK")
Pkg.add("EAGO")
Pkg.add("HiGHS")
Pkg.add("BenchmarkTools")

ENV["CPLEX_STUDIO_BINARIES"] = "/opt/ibm/ILOG/CPLEX_Studio_Community221/cplex/bin/x86-64_linux"
Pkg.add("CPLEX")
Pkg.build("CPLEX")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
    Building CPLEX → `~/.julia/scratchsp

In [2]:
struct Delivery
    x::Float64
    y::Float64
end;
function random_delivery()
    x = rand(0.0:0.1:100.0)
    y = rand(0.0:0.1:100.0)
    return Delivery(x, y)
end;

struct TSProblem
    deliveries::Array{Delivery}
end;

function random_instance(n_deliveries)
    deliveries = [random_delivery() for _=1:n_deliveries]
    problem = TSProblem(deliveries)
    return problem
end;

function dist(del1::Delivery, del2::Delivery)
    return sqrt((del1.x - del2.x)^2 + (del1.y - del2.y)^2)
end
function calc_travelmatrix(deliveries::Array{Delivery})
    tm = zeros(Float64, length(deliveries), length(deliveries))
    for i = 1:length(deliveries)
        for j = 1:length(deliveries)
            tm[i, j] = dist(deliveries[i], deliveries[j])
        end
    end
    return tm
end


calc_travelmatrix (generic function with 1 method)

In [3]:
using JuMP

import GLPK
import EAGO
import HiGHS
import CPLEX

using BenchmarkTools


function basic_mtz(problem, solver, supress_output=false, cplex_silence=false)
    travelmatrix = calc_travelmatrix(problem.deliveries)
    model = Model(solver)
    if supress_output
        set_optimizer_attribute(model, "log_to_console", false)
    end
    if cplex_silence
        set_optimizer_attribute(model, "CPXPARAM_ScreenOutput", 0)
    end
    # route is an adjence matrix representing a route traveled
    route=@variable(model, route[1:length(problem.deliveries), 1:length(problem.deliveries)], Bin)
    # mtzu is a helper variable to ensure no subtours are allowed (only one continous tour)
    # see MTZ constraint
    mtzu = @variable(model, mtzu[1:length(problem.deliveries)], Int)

    # ensure all events are planned
    @constraint(model, [i = 1:length(problem.deliveries)], sum(route[i, :]) == 1.0)
    # ensure there is just one route
    @constraint(model, [c = 1:length(problem.deliveries)], sum(route[:, c]) == 1.0)
    # disallow traveling to itself
    @constraint(model, [j = 1:length(problem.deliveries)], route[j, j] == 0)

    # MTZ constraints for removing subtours
    n = length(problem.deliveries)
    @constraint(model, [ui = 1:n, uj = 2:n], mtzu[ui] + route[ui, uj] <= mtzu[uj]+ (n - 1) * (1 - route[ui, uj]) )

    traveltime = travelmatrix.* route 
    @objective(model, Min, sum(traveltime))
    optimize!(model)
end

problem=random_instance(5)
basic_mtz(problem, EAGO.Optimizer)
basic_mtz(problem, GLPK.Optimizer)
basic_mtz(problem, HiGHS.Optimizer, true)
basic_mtz(problem, CPLEX.Optimizer, false, true)

In [4]:
problem=random_instance(5)
@time  basic_mtz(problem, GLPK.Optimizer)

  0.001175 seconds (3.33 k allocations: 186.484 KiB)


In [5]:
problem=random_instance(10)
@time  basic_mtz(problem, GLPK.Optimizer)

  0.026624 seconds (8.74 k allocations: 598.047 KiB)


In [6]:
@benchmark basic_mtz(p, GLPK.Optimizer) setup=(p=random_instance(10)) evals=3 samples=20 seconds=60

BenchmarkTools.Trial: 20 samples with 3 evaluations.
 Range (min … max):   2.387 ms … 152.069 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     27.636 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   38.124 ms ±  34.890 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃       █    ▃                                                
  █▁▇▇▇▇▁▇█▁▁▇▁█▁▇▇▁▁▁▁▁▁▁▇▁▇▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▁
  2.39 ms         Histogram: frequency by time          152 ms <

 Memory estimate: 598.05 KiB, allocs estimate: 8744.

In [7]:
@benchmark basic_mtz(p, EAGO.Optimizer) setup=(p=random_instance(10)) evals=3 samples=20 seconds=60

BenchmarkTools.Trial: 20 samples with 3 evaluations.
 Range (min … max):   73.660 ms …    1.429 s  ┊ GC (min … max): 0.00% … 0.20%
 Time  (median):     204.720 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   328.637 ms ± 345.277 ms  ┊ GC (mean ± σ):  0.04% ± 0.04%

   █                                                             
  ▅█▅█▁▅▅█▅▁▁▅▁▁▁▁▁▁▁█▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅ ▁
  73.7 ms          Histogram: frequency by time          1.43 s <

 Memory estimate: 1.29 MiB, allocs estimate: 23128.

In [8]:
@benchmark basic_mtz(p, HiGHS.Optimizer, true) setup=(p=random_instance(10)) evals=3 samples=20 seconds=60

BenchmarkTools.Trial: 20 samples with 3 evaluations.
 Range (min … max):    4.908 ms …    1.112 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      51.669 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   172.316 ms ± 283.138 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                              
  █▆▄▄▄▁▄▄▁▁▄▄▁▁▄▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  4.91 ms          Histogram: frequency by time          1.11 s <

 Memory estimate: 962.20 KiB, allocs estimate: 8693.

In [9]:
@benchmark basic_mtz(p, CPLEX.Optimizer, false, true) setup=(p=random_instance(10)) evals=3 samples=20 seconds=60

BenchmarkTools.Trial: 20 samples with 3 evaluations.
 Range (min … max):  11.428 ms … 98.781 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     28.619 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   34.962 ms ± 19.152 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

           █▁▁  ▁    █                                         
  ▆▁▆▁▁▁▆▁▁███▁▁█▁▁▁▁█▁▁▁▁▁▁▆▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  11.4 ms         Histogram: frequency by time        98.8 ms <

 Memory estimate: 662.83 KiB, allocs estimate: 10879.

In [ ]:
problem=random_instance(30)
@time  basic_mtz(problem, GLPK.Optimizer)